[Documentation](https://pymongo.readthedocs.io/en/stable/tutorial.html)


In [1]:
from pymongo import MongoClient
import json


class DB:
    def __init__(self):
        self.client = MongoClient("mongodb://localhost:27017/")
        self.db = self.client["10Academy"]

    def list_collections(self):
        # Returns collections inside the database
        return self.db.list_collection_names()

    def check_if_collection_exist(self, collection_name: str):
        if not self.list_collections().__contains__(collection_name):
            raise Exception(f"Collection, {collection_name} not found.")

    def insert_to_collection(self, collection_name, data):
        self.check_if_collection_exist(collection_name)
        collection = self.db[collection_name]
        return collection.insert_one(data)

    def insert_many_to_collection(self, collection_name, data):
        self.check_if_collection_exist(collection_name)
        result = self.db[collection_name].insert_many(data)
        return result.inserted_ids

    def find_all(self, collection_name):
        self.check_if_collection_exist(collection_name)
        return self.db[collection_name].find()

    def find(self, collection_name, key, value):
        self.check_if_collection_exist(collection_name)
        return self.db[collection_name].find({key: value})
    
    def find_by_id(self, collection_name, _id):
        self.check_if_collection_exist(collection_name)
        return self.db[collection_name].find

    def find_one(self, collection_name, key, value):
        self.check_if_collection_exist(collection_name)
        return self.db[collection_name].find_one({key: value})


In [2]:
class DBWithSchema:
    def __init__(self) -> None:
        self.client = MongoClient("mongodb://localhost:27017/")
        self.db = self.client["10Academy1"]

        self.employee_validator = {
            "$jsonSchema": {
                "bsonType": "object",
                "required": ["name", "age", "company"],
                "properties": {
                    "name": {
                        "bsonType": "string",
                        "description": "must be a string and is required"
                    },
                    "age": {
                        "bsonType": "number",
                        "description": "must be an number and is required"
                    },
                    "company": {
                        "bsonType": "objectId",
                        "description": "must be an objectId and is required"
                    }
                }
            }
        }
        self.company_validator = {
            "$jsonSchema": {
                "bsonType": "object",
                "required": ["name", "country", "city"],
                "properties": {
                    "name": {
                        "bsonType": "string",
                        "description": "must be a string and is required"
                    },
                    "city": {
                        "bsonType": "string",
                        "description": "must be an string and is required"
                    },
                    "country": {
                        "bsonType": "string",
                        "description": "must be an string and is required"
                    }
                }
            }
        }
        try:
            self.db.create_collection("employee")
            self.db.create_collection("company")
        except Exception as e:
            print(e)

        self.db.command("collMod", "employee", validator=self.employee_validator)
        self.db.command("collMod", "company", validator=self.company_validator)

    def list_collections(self):
        return self.db.list_collection_names()
    
    def get_validation(self, collection_name: str) -> dict:
        self.check_if_collection_exist(collection_name)
        return self.db.get_collection(collection_name).options()
    
    def check_if_collection_exist(self, collection_name: str):
        if not self.list_collections().__contains__(collection_name):
            raise Exception(f"Collection, {collection_name} not found.")

    def insert_to_collection(self, collection_name, data):
        self.check_if_collection_exist(collection_name)
        collection = self.db[collection_name]
        return collection.insert_one(data)

    def insert_many_to_collection(self, collection_name, data):
        self.check_if_collection_exist(collection_name)
        result = self.db[collection_name].insert_many(data)
        return result.inserted_ids

    def find_all(self, collection_name):
        self.check_if_collection_exist(collection_name)
        return self.db[collection_name].find()

    def find(self, collection_name, key, value):
        self.check_if_collection_exist(collection_name)
        return self.db[collection_name].find({key: value})
    
    def find_by_id(self, collection_name, _id):
        self.check_if_collection_exist(collection_name)
        return self.db[collection_name].find

    def find_one(self, collection_name, key, value):
        self.check_if_collection_exist(collection_name)
        return self.db[collection_name].find_one({key: value})
    


#### Initialize DB Class

In [3]:
db = DB()

#### Get list of collections

In [4]:
db.list_collections()

['users',
 'slack_data',
 'all-week1-12',
 'all_slack_data',
 'all-week1',
 '10Academy']

#### Insert record to collection

In [5]:
data = {
    "name": "John"
}
inserted_record = db.insert_to_collection('users', data)
print(inserted_record.inserted_id)


6569645e0cd82499b8959322


#### Insert more than record to collection

In [6]:
data = [{
    "name": "Tom"
},
    {
        "name": "Jane",
        "age": 33
}
]
inserted_records = db.insert_many_to_collection('users', data)
print(inserted_records)

[ObjectId('6569645f0cd82499b8959323'), ObjectId('6569645f0cd82499b8959324')]


#### Find All records from collection 

In [7]:
import pprint

records = db.find_all('users')
for record in records:
    pprint.pprint(record)

{'_id': ObjectId('6568390c91b7eef87183c52c'), 'name': 'John'}
{'_id': ObjectId('6568392491b7eef87183c52d'), 'name': 'Tom'}
{'_id': ObjectId('6568392491b7eef87183c52e'), 'age': 33, 'name': 'Jane'}
{'_id': ObjectId('65686fd6d485420a27d08cfd'), 'name': 'John'}
{'_id': ObjectId('65686fded485420a27d08cfe'), 'name': 'Tom'}
{'_id': ObjectId('65686fded485420a27d08cff'), 'age': 33, 'name': 'Jane'}
{'_id': ObjectId('6569645e0cd82499b8959322'), 'name': 'John'}
{'_id': ObjectId('6569645f0cd82499b8959323'), 'name': 'Tom'}
{'_id': ObjectId('6569645f0cd82499b8959324'), 'age': 33, 'name': 'Jane'}


#### Filter record from collection

In [8]:
results = db.find('users', 'name', 'Jane')
for result in results:
    pprint.pprint(result)

{'_id': ObjectId('6568392491b7eef87183c52e'), 'age': 33, 'name': 'Jane'}
{'_id': ObjectId('65686fded485420a27d08cff'), 'age': 33, 'name': 'Jane'}
{'_id': ObjectId('6569645f0cd82499b8959324'), 'age': 33, 'name': 'Jane'}


In [9]:
results = db.find('users', 'name', 'Rix')
for result in results:
    pprint.pprint(result)

#### Find one from collection

In [10]:
result = db.find_one('users', 'name', 'Jane')
pprint.pprint(result)

{'_id': ObjectId('6568392491b7eef87183c52e'), 'age': 33, 'name': 'Jane'}


#### Initialize DBWithSchemaClass

In [11]:
db_with_schema = DBWithSchema()

collection employee already exists


#### View validators

In [12]:
import pprint
company_validator = db_with_schema.get_validation('company')
pprint.pprint(company_validator)

{'validationAction': 'error',
 'validationLevel': 'strict',
 'validator': {'$jsonSchema': {'bsonType': 'object',
                               'properties': {'city': {'bsonType': 'string',
                                                       'description': 'must be '
                                                                      'an '
                                                                      'string '
                                                                      'and is '
                                                                      'required'},
                                              'country': {'bsonType': 'string',
                                                          'description': 'must '
                                                                         'be '
                                                                         'an '
                                                                         'string '
        

In [13]:
employee_validator = db_with_schema.get_validation('employee')
pprint.pprint(employee_validator)

{'validationAction': 'error',
 'validationLevel': 'strict',
 'validator': {'$jsonSchema': {'bsonType': 'object',
                               'properties': {'age': {'bsonType': 'number',
                                                      'description': 'must be '
                                                                     'an '
                                                                     'number '
                                                                     'and is '
                                                                     'required'},
                                              'company': {'bsonType': 'objectId',
                                                          'description': 'must '
                                                                         'be '
                                                                         'an '
                                                                         'objectId '
          

#### Create Company

In [14]:
company_data = [
  {
    "name": "Mercedes-Benz",
    "city": "Stuttgart",
    "country": "Germany"
  },
  {
    "name": "Chevrolet",
    "city": "Detroit",
    "country": "United States"
  }
]
inserted_records = db_with_schema.insert_many_to_collection('company', company_data)
print(inserted_records)

[ObjectId('656964690cd82499b8959326'), ObjectId('656964690cd82499b8959327')]


In [15]:
employee_data = [
  {
    "name": "Jane",
    "age": 24,
    "company": inserted_records[0]
  },
  {
    "name": "John",
    "age": 26,
    "company": inserted_records[1]
  },
  {
    "name": "Amy",
    "age": 23,
    "company": inserted_records[1]
  },
  {
    "name": "Jack",
    "age": 22,
    "company": inserted_records[0]
  },
]
inserted_employees = db_with_schema.insert_many_to_collection('employee', employee_data)
print(inserted_employees)

[ObjectId('6569646a0cd82499b8959328'), ObjectId('6569646a0cd82499b8959329'), ObjectId('6569646a0cd82499b895932a'), ObjectId('6569646a0cd82499b895932b')]


#### Select all Employees

In [16]:
import pprint

employees = db_with_schema.find_all('employee')
for employee in employees:
    pprint.pprint(employee)

{'_id': ObjectId('6568395b91b7eef87183c532'),
 'age': 24,
 'company': ObjectId('6568395a91b7eef87183c530'),
 'name': 'Jane'}
{'_id': ObjectId('6568395b91b7eef87183c533'),
 'age': 26,
 'company': ObjectId('6568395a91b7eef87183c531'),
 'name': 'John'}
{'_id': ObjectId('6568395b91b7eef87183c534'),
 'age': 23,
 'company': ObjectId('6568395a91b7eef87183c531'),
 'name': 'Amy'}
{'_id': ObjectId('6568395b91b7eef87183c535'),
 'age': 22,
 'company': ObjectId('6568395a91b7eef87183c530'),
 'name': 'Jack'}
{'_id': ObjectId('65687016d485420a27d08d03'),
 'age': 24,
 'company': ObjectId('65687015d485420a27d08d01'),
 'name': 'Jane'}
{'_id': ObjectId('65687016d485420a27d08d04'),
 'age': 26,
 'company': ObjectId('65687015d485420a27d08d02'),
 'name': 'John'}
{'_id': ObjectId('65687016d485420a27d08d05'),
 'age': 23,
 'company': ObjectId('65687015d485420a27d08d02'),
 'name': 'Amy'}
{'_id': ObjectId('65687016d485420a27d08d06'),
 'age': 22,
 'company': ObjectId('65687015d485420a27d08d01'),
 'name': 'Jack'}
{'

#### Look UP

In [17]:
lookup = [
    {
        "$lookup": {
            "from": "employee",
            "localField": "_id",
            "foreignField": "company",
            "as": "employees"
        }
    }
]
employees_with_company = db_with_schema.db.company.aggregate(lookup)
employees_list = list(employees_with_company)
for employee in employees_list:
    pprint.pprint(employee)

{'_id': ObjectId('6568395a91b7eef87183c530'),
 'city': 'Stuttgart',
 'country': 'Germany',
 'employees': [{'_id': ObjectId('6568395b91b7eef87183c532'),
                'age': 24,
                'company': ObjectId('6568395a91b7eef87183c530'),
                'name': 'Jane'},
               {'_id': ObjectId('6568395b91b7eef87183c535'),
                'age': 22,
                'company': ObjectId('6568395a91b7eef87183c530'),
                'name': 'Jack'}],
 'name': 'Mercedes-Benz'}
{'_id': ObjectId('6568395a91b7eef87183c531'),
 'city': 'Detroit',
 'country': 'United States',
 'employees': [{'_id': ObjectId('6568395b91b7eef87183c533'),
                'age': 26,
                'company': ObjectId('6568395a91b7eef87183c531'),
                'name': 'John'},
               {'_id': ObjectId('6568395b91b7eef87183c534'),
                'age': 23,
                'company': ObjectId('6568395a91b7eef87183c531'),
                'name': 'Amy'}],
 'name': 'Chevrolet'}
{'_id': ObjectId('656

importing json folder messages in to mongodb

In [29]:
import os

def import_data_from_json(db, collection_name, json_folder):
    """
    Import data from JSON files in a folder into the specified collection.

    Parameters:
    - db (pymongo.database.Database): The MongoDB database object.
    - collection_name (str): The name of the MongoDB collection.
    - json_folder (str): The path to the folder containing JSON files.

    Returns:
    - list: List of inserted document IDs.
    """
    # List all JSON files in the folder
    json_files = [f for f in os.listdir(json_folder) if f.endswith('.json')]

    # List to store inserted document IDs
    inserted_ids_list = []

    # Iterate over JSON files
    for json_file in json_files:
        # Construct the full path to the JSON file
        json_file_path = os.path.join(json_folder, json_file)

        # Read data from the JSON file
        with open(json_file_path, 'r') as file:
            data = json.load(file)

        # Insert data into the collection
        result = db[collection_name].insert_many(data)

        # Add inserted document IDs to the list
        inserted_ids_list.extend(result.inserted_ids)

    return inserted_ids_list

# Connect to the local MongoDB instance
client = MongoClient("mongodb://localhost:27017/")

# Choose or create a database
db = client["10Academy"]

# Specify the collection name
collection_name = "all-week1-12"

# Specify the path to the folder containing JSON files
json_folder_path =  '../data/all-week12'

# Import data from JSON files in the folder
inserted_ids = import_data_from_json(db, collection_name, json_folder_path)

# Print the inserted document IDs
print("Inserted IDs:", inserted_ids)

# Close the MongoDB connection
client.close()


Inserted IDs: [ObjectId('656964ec0cd82499b895a340'), ObjectId('656964ec0cd82499b895a341'), ObjectId('656964ec0cd82499b895a342'), ObjectId('656964ec0cd82499b895a343'), ObjectId('656964ec0cd82499b895a344'), ObjectId('656964ec0cd82499b895a345'), ObjectId('656964ec0cd82499b895a346'), ObjectId('656964ec0cd82499b895a347'), ObjectId('656964ec0cd82499b895a348'), ObjectId('656964ec0cd82499b895a349'), ObjectId('656964ec0cd82499b895a34a'), ObjectId('656964ec0cd82499b895a34b'), ObjectId('656964ec0cd82499b895a34c'), ObjectId('656964ec0cd82499b895a34d'), ObjectId('656964ec0cd82499b895a34e'), ObjectId('656964ec0cd82499b895a34f'), ObjectId('656964ec0cd82499b895a350'), ObjectId('656964ec0cd82499b895a351'), ObjectId('656964ec0cd82499b895a352'), ObjectId('656964ec0cd82499b895a353'), ObjectId('656964ec0cd82499b895a354'), ObjectId('656964ec0cd82499b895a355'), ObjectId('656964ec0cd82499b895a356'), ObjectId('656964ec0cd82499b895a357'), ObjectId('656964ec0cd82499b895a358'), ObjectId('656964ec0cd82499b895a359'